# Домашняя работа 

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов 

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("ds_salaries.csv")

In [3]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [180]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer


y = df.salary_in_usd
X = df.drop(['salary', 'salary_in_usd'], axis= 1)
X_train, X_test_val, y_train, y_test_val = train_test_split(
    X, y, test_size=0.2, random_state=123
)
X_val, X_test, y_val, y_test = train_test_split(
    X_test_val, y_test_val, test_size=0.5, random_state=123
)

## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

cat_features_train = X_train.dtypes == "object"
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'), cat_features_train),
    remainder='passthrough')
transformed_train = transformer.fit_transform(X_train).toarray()
X_train_l = pd.DataFrame(
    transformed_train, 
    columns=transformer.get_feature_names_out())

cat_features_val = X_val.dtypes == "object"

transformed_val = transformer.transform(X_val).toarray()
X_val_l = pd.DataFrame(
    transformed_val, 
    columns=transformer.get_feature_names_out())

cat_features_test = X_test.dtypes == "object"
transformed_test = transformer.transform(X_test).toarray()
X_test_l = pd.DataFrame(
    transformed_test, 
    columns=transformer.get_feature_names_out())

ss = StandardScaler()
X_train_l = ss.fit_transform(X_train_l)
X_val_l = ss.transform(X_val_l)
X_test_l = ss.transform(X_test_l)

lr = LinearRegression().fit(X_train_l, y_train)
print(f"Validation RMSE: {mean_squared_error(y_val, lr.predict(X_val_l), squared=False)}")
print(f"Test RMSE: {mean_squared_error(y_test, lr.predict(X_test_l), squared=False)}")
print(f"Validation MAPE: {mean_absolute_percentage_error(y_val, lr.predict(X_val_l))}")
print(f"Test MAPE: {mean_absolute_percentage_error(y_test, lr.predict(X_test_l))}")

Validation RMSE: 3.7866249129416595e+17
Test RMSE: 6.023480300609078e+17
Validation MAPE: 2233289783216.386
Test MAPE: 659137930373.1843


## Задание 3 (0.5 балла)

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [36]:
import lightgbm as lgb
import sklearn

cat_features_train = X_train.dtypes == "object"
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'), cat_features_train),
    remainder='passthrough')
transformed_train = transformer.fit_transform(X_train).toarray()
X_train_l = pd.DataFrame(
    transformed_train, 
    columns=transformer.get_feature_names_out())

cat_features_val = X_val.dtypes == "object"

transformed_val = transformer.transform(X_val).toarray()
X_val_l = pd.DataFrame(
    transformed_val, 
    columns=transformer.get_feature_names_out())

cat_features_test = X_test.dtypes == "object"
transformed_test = transformer.transform(X_test).toarray()
X_test_l = pd.DataFrame(
    transformed_test, 
    columns=transformer.get_feature_names_out())

train_dataset = lgb.Dataset(X_train_l, y_train, feature_name=X_train_l.columns.values.tolist())
val_dataset = lgb.Dataset(X_val_l, y_val, feature_name=X_val_l.columns.values.tolist())
test_dataset = lgb.Dataset(X_test_l, y_test, feature_name=X_test_l.columns.values.tolist())

booster = lgb.train({"objective": "regression"},
                    train_set=train_dataset, valid_sets=(val_dataset,),
                    num_boost_round=10)

train_preds = booster.predict(X_train_l)
val_preds = booster.predict(X_val_l)
test_preds = booster.predict(X_test_l)

print(f"Validation RMSE: {mean_squared_error(y_val, val_preds, squared=False)}")
print(f"Test RMSE: {mean_squared_error(y_test, test_preds, squared=False)}")
print(f"Validation MAPE: {mean_absolute_percentage_error(y_val, val_preds)}")
print(f"Test MAPE: {mean_absolute_percentage_error(y_test, test_preds)}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.045913 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 3004, number of used features: 43
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 138580.205393
[1]	valid_0's l2: 3.38447e+09
[2]	valid_0's l2: 3.09522e+09
[3]	valid_0's l2: 2.86948e+09
[4]	valid_0's l2: 2.68197e+09
[5]	valid_0's l2: 2.52952e+09
[6]	valid_0's l2: 2.40706e+09
[7]	valid_0's l2: 2.31077e+09
[8]	valid_0's l2: 2.22406e+09
[9]	valid_0's l2: 2.16576e+09
[10]	valid_0's l2: 2.11226e+09
Validation RMSE: 45959.38206922794
Test RMSE: 46869.179348244725
Validation MAPE: 0.5223231060550186
Test MAPE: 0.43817998656238

## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [166]:
# one-hot encoding
cat_features_train = X_train.dtypes == "object"
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'), cat_features_train[:3]),
    remainder='passthrough')
transformed_train = transformer.fit_transform(X_train[["work_year", "experience_level", "employment_type"]])
X_train_l = pd.DataFrame(
    transformed_train, 
    columns=transformer.get_feature_names_out())

transformed_val = transformer.transform(X_val[["work_year", "experience_level", "employment_type"]])
X_val_l = pd.DataFrame(
    transformed_val, 
    columns=transformer.get_feature_names_out())

transformed_test = transformer.transform(X_test[["work_year", "experience_level", "employment_type"]])
X_test_l = pd.DataFrame(
    transformed_test, 
    columns=transformer.get_feature_names_out())

#labelencoder
from sklearn import preprocessing
cat_features= ["salary_currency", "company_size"]
train_val = X_train.append(X_val)
encoders = dict()

for cat in cat_features:
    encoders[cat] = preprocessing.LabelEncoder()
    train_val[cat] = encoders[cat].fit(train_val[cat])
    X_train_l[cat] = encoders[cat].transform(X_train[cat])
    X_val_l[cat] = encoders[cat].transform(X_val[cat])
    X_test_l[cat] = encoders[cat].transform(X_test[cat])

#mean target encoding    
import category_encoders as ce

encoder = ce.TargetEncoder()
X_train_new = encoder.fit_transform(X_train[["job_title", "employee_residence"]], y_train)
X_val_new = encoder.transform(X_val[["job_title", "employee_residence"]])
X_test_new = encoder.transform(X_test[["job_title", "employee_residence"]])

X_train_l = pd.concat([X_train_l, X_train_new.reset_index(drop=True)], axis=1)
X_val_l = pd.concat([X_val_l, X_val_new.reset_index(drop=True)], axis=1)
X_test_l = pd.concat([X_test_l, X_test_new.reset_index(drop=True)], axis=1)

/tmp/ipykernel_95/1972532779.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_val = X_train.append(X_val)


In [168]:
#Linear regression
ss = StandardScaler()
X_train_lr = ss.fit_transform(X_train_l)
X_val_lr = ss.transform(X_val_l)
X_test_lr = ss.transform(X_test_l)

lr = LinearRegression().fit(X_train_l, y_train)
print(f"Validation RMSE lr: {mean_squared_error(y_val, lr.predict(X_val_l), squared=False)}")
print(f"Test RMSE lr: {mean_squared_error(y_test, lr.predict(X_test_l), squared=False)}")
print(f"Validation MAPE lr: {mean_absolute_percentage_error(y_val, lr.predict(X_val_l))}")
print(f"Test MAPE lr: {mean_absolute_percentage_error(y_test, lr.predict(X_test_l))}")

Validation RMSE lr: 44498.913242987146
Test RMSE lr: 46380.28748273
Validation MAPE lr: 0.4301482909298643
Test MAPE lr: 0.36652514925802804


In [169]:
#lightgbm
train_dataset = lgb.Dataset(X_train_l, y_train, feature_name=X_train_l.columns.values.tolist())
val_dataset = lgb.Dataset(X_val_l, y_val, feature_name=X_val_l.columns.values.tolist())
test_dataset = lgb.Dataset(X_test_l, y_test, feature_name=X_test_l.columns.values.tolist())

booster = lgb.train({"objective": "regression"},
                    train_set=train_dataset, valid_sets=(val_dataset,),
                    num_boost_round=10)

train_preds = booster.predict(X_train_l)
val_preds = booster.predict(X_val_l)
test_preds = booster.predict(X_test_l)

print(f"Validation RMSE lightgbm: {mean_squared_error(y_val, val_preds, squared=False)}")
print(f"Test RMSE lightgbm: {mean_squared_error(y_test, test_preds, squared=False)}")
print(f"Validation MAPE lightgbm: {mean_absolute_percentage_error(y_val, val_preds)}")
print(f"Test MAPE lightgbm: {mean_absolute_percentage_error(y_test, test_preds)}")

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016203 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 142
[LightGBM] [Info] Number of data points in the train set: 3004, number of used features: 10
[LightGBM] [Info] Start training from score 138580.205393
[1]	valid_0's l2: 3.4028e+09
[2]	valid_0's l2: 3.13168e+09
[3]	valid_0's l2: 2.91194e+09
[4]	valid_0's l2: 2.74337e+09
[5]	valid_0's l2: 2.60255e+09
[6]	valid_0's l2: 2.48527e+09
[7]	valid_0's l2: 2.37995e+09
[8]	valid_0's l2: 2.29298e+09
[9]	valid_0's l2: 2.22004e+09
[10]	valid_0's l2: 2.16605e+09
Validation RMSE lightgbm: 46540.89003056658
Test RMSE lightgbm: 47186.01613261466
Validation MAPE lightgbm: 0.5546917011622176
Test MAPE lightgbm: 0.44561617175276635


### Ответ:
Линейная регрессия справилась лучше, так как для lightgbm нужно я не подбирала гиперпараметры, чтобы повысить точность. 
Вообще один из выводов - это то, что удобнее всего использовать TargetEncoder, потому что иначе может появиться ошибка, если в тренировочных данных не было той категории, которая была в тестовых данных. 

## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [188]:
y_new = pd.DataFrame(y_test)
y_new['predictions'] = test_preds
y_new['difference'] = abs(y_new['predictions'] - y_new['salary_in_usd'])

In [197]:
the_highest_error = y_new.sort_values('difference', ascending=False)[:20].index.values

In [199]:
X_test.loc[the_highest_error]

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
2374,2022,SE,FT,Data Scientist,USD,US,100,US,M
33,2023,SE,FT,Computer Vision Engineer,USD,US,0,US,M
1286,2023,SE,FT,Machine Learning Engineer,USD,US,100,US,M
1458,2023,SE,FT,Data Engineer,USD,US,0,US,M
841,2023,MI,FT,Data Engineer,USD,US,0,US,M
1185,2023,SE,FT,Machine Learning Engineer,USD,US,100,US,M
2986,2022,SE,FT,ETL Developer,USD,US,0,US,M
565,2023,SE,FT,Data Scientist,USD,US,100,US,M
2595,2022,SE,FT,Data Scientist,USD,US,100,US,M
2591,2022,SE,FT,Data Scientist,USD,US,100,US,M


Я проверила, что категории "HN" нет в тренировочных и валидационных данных, это могло послужить одной из причин высокого значения ошибки. Однако по остальным параметрам сложно сделать вывод, почему ошибка оказалась большой.

## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [46]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 4.9 MB/s eta 0:00:0000:0100:01


In [50]:
import xgboost as xgb

params = {
    "objective": "reg:squarederror",
    "n_estimators":100,
    "max_depth": 4,
    "learning_rate": 0.01,
    "subsample": 0.8,
    "colsample_bytree": 0.9,
    "colsample_bylevel": 0.8,
    "reg_lambda": 0.1,
    "eval_metric": "rmse",
    "random_state": 42,
}

reg = xgb.XGBRegressor(**params)

%time
reg.fit(X_train_l, 
        y_train, 
        verbose=False,
        eval_set= [(X_train_l, y_train), (X_val_l, y_val)],
        early_stopping_rounds= 3
        )
results = reg.evals_result()
best_iter = reg.best_iteration
best_iter
print(results['validation_1']['rmse'][best_iter])

%time
test_preds = reg.predict(X_test_l)
val_preds = reg.predict(X_val_l)
print(f"Validation RMSE: {mean_squared_error(y_val, val_preds, squared=False)}")
print(f"Test RMSE: {mean_squared_error(y_test, test_preds, squared=False)}")
print(f"Validation MAPE: {mean_absolute_percentage_error(y_val, val_preds)}")
print(f"Test MAPE: {mean_absolute_percentage_error(y_test, test_preds)}")


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15.5 µs
66517.4501198661
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.54 µs
Validation RMSE: 66517.44996999443
Test RMSE: 67804.26649888395
Validation MAPE: 0.39997019887202667
Test MAPE: 0.3773581608845906


## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [45]:
import lightgbm as lgb
import sklearn

cat_features_train = X_train.dtypes == "object"
transformer = make_column_transformer(
    (OneHotEncoder(handle_unknown = 'ignore'), cat_features_train),
    remainder='passthrough')
transformed_train = transformer.fit_transform(X_train).toarray()
X_train_l = pd.DataFrame(
    transformed_train, 
    columns=transformer.get_feature_names_out())

cat_features_val = X_val.dtypes == "object"

transformed_val = transformer.transform(X_val).toarray()
X_val_l = pd.DataFrame(
    transformed_val, 
    columns=transformer.get_feature_names_out())

cat_features_test = X_test.dtypes == "object"
transformed_test = transformer.transform(X_test).toarray()
X_test_l = pd.DataFrame(
    transformed_test, 
    columns=transformer.get_feature_names_out())

train_dataset = lgb.Dataset(X_train_l, y_train, feature_name=X_train_l.columns.values.tolist())
val_dataset = lgb.Dataset(X_val_l, y_val, feature_name=X_val_l.columns.values.tolist())
test_dataset = lgb.Dataset(X_test_l, y_test, feature_name=X_test_l.columns.values.tolist())

%time
booster = lgb.train({"objective": "regression", "num_iterations": 100, "learning_rate": 0.1, "max_depth": 20},
                        train_set=train_dataset, valid_sets=(val_dataset,),
                        num_boost_round=10)
%time
train_preds = booster.predict(X_train_l)
%time
val_preds = booster.predict(X_val_l)
%time
test_preds = booster.predict(X_test_l)

print(f"Validation RMSE: {mean_squared_error(y_val, val_preds, squared=False)}")
print(f"Test RMSE: {mean_squared_error(y_test, test_preds, squared=False)}")
print(f"Validation MAPE: {mean_absolute_percentage_error(y_val, val_preds)}")
print(f"Test MAPE: {mean_absolute_percentage_error(y_test, test_preds)}")

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000357 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 3004, number of used features: 43
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training 

/home/arinkazam/.local/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания

### Ответ:
Из двух моделей, которые я использовала (lightgbm, xgboost) лучше себя показал lightgbm, он и обучается быстрее, и качество выдал выше.